<a href="https://colab.research.google.com/github/alfa7g7/Analisis-Cuantitativo-1er-semestre/blob/main/Examen_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FACULTAD DE INGENIERÍA, DISEÑO Y CIENCIAS APLICADAS MAESTRÍA EN CIENCIA DE DATOS TIC 60153 – Análisis Cuantitativo Examen 2 (50 %): Modelos de Regularización

### Ejercicio 1. 1)	Considere el conjunto de datos “Boston Housing Data” presentados en Harrison and Rubinfeld (1978). Defina como variable respuesta a la columna MEDV. Realice una partición 80-20, donde el primer 80 % de los datos son datos de entrenamiento y el restante 20 % son datos para prueba.


- Genere los modelos de regresión por regularización **Ridge, LASSO y Elastic-Net** para los datos de entrenamiento. Encuentre los valores óptimos de $α$ y $λ$ junto a su respectiva gráfica de evolución de los coeficientes de regresión. Compare los modelos en términos de la selección de variables, interprete los coeficientes y escriba la ecuación ajustada de regresión para cada caso. Finalmente, realice una predicción con las observaciones de prueba y determine cual de los tres modelos es el mejor en capacidad predictiva **(RMSE)**.
- Sea $Z ∈ \mathbb{R}^{n \times p}$ una muestra de datos multivariantes. La distancia de Mahalanobis (MD), presentado por Mahalanobis (1936) y conocida como la distancia estadística, se define de la siguiente forma:
$$MD(zi) = \sqrt{(zi − \hatµ)'\hat{∑}^{-1}(zi - \hat{µ})},\  para\ i=1, . . . , n.$$
Donde $xi ∈ \mathbb{R}^{p\times1}$ denota una observación (fila), pero transpuesta, de la muestra, $\hat{µ} ∈ \mathbb{R}^{p\times1}$ el vector de medias (vector con los promedios de cada columna o variable) y $\hat{Σ}^{−1} ∈ \mathbb{R}^{p\times p}$ la inversa de la matriz de covarianza de los datos. MD puede interpretarse como la distancia
que tiene cada individuo de la muestra al individuo ideal o equilibrio (representado por $\hat{µ}$), considerando la estructura de dependencia de las variables.

  Esta distancia tiene múltiples usos, como por ejemplo la detección de observaciones atípicas. Se dice que una observación $zi$ es atípica sí $MD(zi)> \chi^{2}_{(q,p)},$ donde q denota un percentil, usualmente $q ∈ [0.95, 0.99]$, de la distribución Chi-squared de parámetro $p$.
  
  De acuerdo a la información anterior: Detecte las observaciones atípicas multivariantes de la muestra, usando $q = 0.90, 0.95, 0.975, 0.99$ y en un gráfico muestre las distancias calculadas para cada individuo y los puntos de corte $(\chi^{2}_{(q,p)})$ ¿Qué puede concluir acerca de los individuos atípicos para cada punto de corte? ¿Quienes son? ¿El gráfico de distancias muestra alguna observación NO atípica que para usted si lo fue? ¿Qué puede concluir acerca del comportamiento de q en el valor del punto de corte? Justifique detalladamente.

- Realice dos modelos de regresión Robusto–Regularizado, combinando la técnica de detección de atípicos multivariantes más la regularización Elastic-Net. Encuentre los valores óptimos de $α^{*}$ y $λ^{*}$, interprete los coeficientes de regresión, escriba la respectiva ecuación de regresión resultante y valide los supuestos del modelo $(ε_i \ iid∼   N(0, σ^{2}))$. Finalmente, realice una predicción con las observaciones de prueba y determine el **RMSE** de estos dos modelos. Compare los resultados con los modelos estimados en el ítem 1.

  NOTA: La aplicación de la técnica de detección de atípicos debe realizarla de la siguiente manera:
  
  - Modelo 1: Estandarizar las variables y calcular MD usando el vector de medianas en vez del vector de medias y la matriz de correlación de Spearman en vez de la matriz de covarianzas usual. Luego, eliminar los atípicos y con la muestra limpia estimar el modelo regularizado.

  - Modelo 2: Estandarizar las variables y calcular MD usando el vector de medianas en vez del vector de medias y la matriz de correlación de Spearman en vez de la matriz de covarianzas usual. Luego, determinar como observaciones NO atípicas los primeros r n+p+1 l individuos con las distancias más pequeñas. Finalmente, con la muestra limpia estimar el modelo regularizado.
  
  Ejemplo: Si $n = 1000$ y $p = 100$ entonces r n+p+1 l = r 1000+100+1 l = r550.5l = 551. 2	2 Es decir, debe identificar como NO atípicos los 551 individuos de la muestra con las distancias de Mahalanobis más pequeñas.


#### Librerias a importar

In [99]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.utils import shuffle
from itertools import combinations

#### Importación del conjunto de datos Boston

**Nota:**  
El dataset de boston ha sido removido de sklearn.datasets, su importación se ha cancelado por un tema de racismo. Usamos su versión original y agregamos el enlace de la misma. Al final también elimnamos la variable que hace alusión a este tema
[Fuente del conjunto de datos](https://lib.stat.cmu.edu/datasets/boston)

In [149]:
# Carga del conjunto de datos reemplazando la forma deshabilitada por ser racista desde la fuente original
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None,
  names=["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV"])
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

# Acomodando un dataframe df con todo el conjunto de datos y sus nombres de columnas
df = pd.DataFrame(data)
df = df.assign(target=target)
df = df.drop(columns=[11, 12, 13])
df.columns = ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV"]

#NO al racismo jamás, quitamos la columna que hace referencia al estudio racista y sin ella trabajaremos.
df = df.drop(columns='B')

#### Defina como variable respuesta a la columna MEDV. Realice una partición 80-20, donde el primer 80 % de los datos son datos de entrenamiento y el restante 20 % son datos para prueba.

In [154]:
# definimos también una semilla de random_state = 1 para que la partición cada vez que se corra sea la misma.
(X_training, X_test, y_training, y_test) = train_test_split(df.drop(columns="MEDV"), df.MEDV, train_size=0.8, random_state=1)

In [155]:
# Verificación de la distribución de 80% entrenamiento(training) y 20% prueba(test)
len( X_training ), len( X_test ), len( y_training ), len( y_test )

(404, 102, 404, 102)

In [156]:
len(df)

506

In [159]:
training_percentage = (len(X_training) * 100) / len(df)
print(f"Training data percentage: {training_percentage:.2f}%")

Training data percentage: 79.84%
